In [73]:
#importing the necessary requirement
import requests
import json
import pandas as pd
import random
from datetime import datetime, timedelta
from tabulate import tabulate

In [34]:
# List of Nigerian cities with latitude & longitude
CITIES = [
    {"name": "Lagos", "lat": 6.5244, "lon": 3.3792},
    {"name": "Abuja", "lat": 9.0579, "lon": 7.4951},
    {"name": "Kano", "lat": 12.0022, "lon": 8.5919},
    {"name": "Ibadan", "lat": 7.3775, "lon": 3.9470},
    {"name": "Port Harcourt", "lat": 4.8156, "lon": 7.0498},
    {"name": "Benin City", "lat": 6.3382, "lon": 5.6250},
    {"name": "Maiduguri", "lat": 11.8464, "lon": 13.1603},
    {"name": "Zaria", "lat": 11.0855, "lon": 7.7190},
    {"name": "Aba", "lat": 5.1082, "lon": 7.3667},
    {"name": "Jos", "lat": 9.8965, "lon": 8.8583},
    {"name": "Ilorin", "lat": 8.4960, "lon": 4.5421},
    {"name": "Oyo", "lat": 7.8468, "lon": 3.9271},
    {"name": "Enugu", "lat": 6.5244, "lon": 7.5186},
    {"name": "Abeokuta", "lat": 7.1557, "lon": 3.3452},
    {"name": "Onitsha", "lat": 6.1413, "lon": 6.8022},
    {"name": "Kaduna", "lat": 10.5244, "lon": 7.4383},
    {"name": "Owerri", "lat": 5.4853, "lon": 7.0355},
    {"name": "Uyo", "lat": 5.0370, "lon": 7.9128},
    {"name": "Calabar", "lat": 4.9826, "lon": 8.3346},
    {"name": "Osogbo", "lat": 7.7676, "lon": 4.5564},
    {"name": "Sokoto", "lat": 13.0059, "lon": 5.2476},
    {"name": "Minna", "lat": 9.6152, "lon": 6.5463},
    {"name": "Makurdi", "lat": 7.7335, "lon": 8.5212},
    {"name": "Bauchi", "lat": 10.3142, "lon": 9.8463},
    {"name": "Akure", "lat": 7.2526, "lon": 5.1931},
    # Add more cities if needed...
]

# Define date range (January 1, 2024, to December 31, 2024)
start_date = "2023-01-01"
end_date = "2024-12-31"

# Store collected data
weather_data_list = []

# Loop through each city
for city in CITIES:
    city_name = city["name"]
    lat = city["lat"]
    lon = city["lon"]

    # Open-Meteo API URL with hourly data
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,precipitation,wind_speed_10m,weathercode,cloudcover,relative_humidity_2m,surface_pressure&timezone=Africa/Lagos"

    # Fetch data
    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()
        timestamps = weather_data["hourly"]["time"]

        # Extract weather details for each hour
        for i in range(len(timestamps)):
            weather_data_list.append({
                "city": city_name,
                "latitude": lat,
                "longitude": lon,
                "datetime": timestamps[i],  # Hourly datetime
                "temperature": weather_data["hourly"]["temperature_2m"][i],
                "precipitation": weather_data["hourly"]["precipitation"][i],  # Hourly rainfall (mm)
                "wind_speed": weather_data["hourly"]["wind_speed_10m"][i],
                "weather_code": weather_data["hourly"]["weathercode"][i],  # Weather condition code
                "cloud_coverage": weather_data["hourly"]["cloudcover"][i],  # Cloud cover (%)
                "humidity": weather_data["hourly"]["relative_humidity_2m"][i],  # Humidity (%)
                "pressure": weather_data["hourly"]["surface_pressure"][i],  # Pressure (hPa)
            })

    else:
        print(f"Failed to fetch data for {city_name}. Status Code: {response.status_code}")

# Convert collected data to DataFrame
df = pd.DataFrame(weather_data_list)

In [37]:
# Display first few rows
df.head()

,city,latitude,longitude,datetime,temperature,precipitation,wind_speed,weather_code,cloud_coverage,humidity,pressure
0,Lagos,6.5244,3.3792,2023-01-01T00:00,25.9,0.0,3.7,0,0,71,1010.6
1,Lagos,6.5244,3.3792,2023-01-01T01:00,23.2,0.0,10.2,0,0,80,1010.8
2,Lagos,6.5244,3.3792,2023-01-01T02:00,22.4,0.0,11.7,0,0,73,1010.2
3,Lagos,6.5244,3.3792,2023-01-01T03:00,21.8,0.0,10.7,0,0,62,1009.6
4,Lagos,6.5244,3.3792,2023-01-01T04:00,21.5,0.0,8.5,0,13,59,1009.3


In [38]:
df.describe()

,latitude,longitude,temperature,precipitation,wind_speed,weather_code,cloud_coverage,humidity,pressure
count,438600.000000,438600.000000,438600.000000,438600.000000,438600.000000,438600.000000,438600.000000,438600.000000,438600.000000
mean,8.077396,6.798384,26.511361,0.128062,7.912297,11.209920,69.891347,66.926685,979.299329
std,2.363307,2.218351,4.306777,0.671438,3.961111,19.744687,38.654133,27.664001,29.793528
min,4.815600,3.345200,11.000000,0.000000,0.000000,0.000000,0.000000,3.000000,872.000000
25%,6.338200,5.193100,23.600000,0.000000,4.900000,1.000000,34.000000,45.000000,970.400000
50%,7.733500,7.049800,25.800000,0.000000,7.400000,3.000000,96.000000,75.000000,983.400000
75%,9.896500,7.912800,29.100000,0.000000,10.300000,3.000000,100.000000,91.000000,1003.100000
max,13.005900,13.160300,44.300000,30.700000,34.600000,65.000000,100.000000,100.000000,1017.800000


In [39]:
df.precipitation.value_counts()

precipitation
0.0     359130
0.1      32764
0.2      12614
0.3       6189
0.4       4033
         ...  
20.7         1
20.4         1
15.2         1
18.9         1
30.4         1
Name: count, Length: 182, dtype: int64

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438600 entries, 0 to 438599
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   city            438600 non-null  object 
 1   latitude        438600 non-null  float64
 2   longitude       438600 non-null  float64
 3   datetime        438600 non-null  object 
 4   temperature     438600 non-null  float64
 5   precipitation   438600 non-null  float64
 6   wind_speed      438600 non-null  float64
 7   weather_code    438600 non-null  int64  
 8   cloud_coverage  438600 non-null  int64  
 9   humidity        438600 non-null  int64  
 10  pressure        438600 non-null  float64
dtypes: float64(6), int64(3), object(2)
memory usage: 36.8+ MB


In [55]:
#converting date object to a datetime format 
df['datetime'] = pd.to_datetime(df['datetime'])

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438600 entries, 0 to 438599
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   city            438600 non-null  object        
 1   latitude        438600 non-null  float64       
 2   longitude       438600 non-null  float64       
 3   datetime        438600 non-null  datetime64[ns]
 4   temperature     438600 non-null  float64       
 5   precipitation   438600 non-null  float64       
 6   wind_speed      438600 non-null  float64       
 7   weather_code    438600 non-null  int64         
 8   cloud_coverage  438600 non-null  int64         
 9   humidity        438600 non-null  int64         
 10  pressure        438600 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 36.8+ MB


In [66]:
# Save to CSV
df.to_csv("nigeria_weather_hourly_2023&2024.csv", index=False)

In [70]:
# Define the data dictionary with correct variables
data_dict = [
    {"Variable": "city", "Type": "String", "Unit": "N/A", "Description": "Name of the city where the weather data was collected."},
    {"Variable": "latitude", "Type": "Float", "Unit": "Degrees", "Description": "Latitude coordinate of the city."},
    {"Variable": "longitude", "Type": "Float", "Unit": "Degrees", "Description": "Longitude coordinate of the city."},
    {"Variable": "datetime", "Type": "Datetime", "Unit": "UTC (ISO 8601)", "Description": "Timestamp of the weather observation in hourly intervals."},
    {"Variable": "temperature", "Type": "Float", "Unit": "°C", "Description": "Air temperature measured at 2 meters above the ground."},
    {"Variable": "precipitation", "Type": "Float", "Unit": "mm", "Description": "Amount of rainfall recorded per hour."},
    {"Variable": "wind_speed", "Type": "Float", "Unit": "m/s", "Description": "Wind speed measured at 10 meters above the ground."},
    {"Variable": "weather_code", "Type": "Integer", "Unit": "N/A", "Description": "Coded representation of weather conditions (e.g., clear sky, rain, snow)."},
    {"Variable": "cloud_coverage", "Type": "Float", "Unit": "%", "Description": "The percentage of sky covered by clouds."},
    {"Variable": "humidity", "Type": "Float", "Unit": "%", "Description": "Relative humidity at 2 meters above the ground."},
    {"Variable": "pressure", "Type": "Float", "Unit": "hPa", "Description": "Surface atmospheric pressure in hectopascals (hPa)."},
]

# Convert to a DataFrame
df = pd.DataFrame(data_dict)

In [71]:
df.head()

,Variable,Type,Unit,Description
0,city,String,N/A,Name of the city where the weather data was co...
1,latitude,Float,Degrees,Latitude coordinate of the city.
2,longitude,Float,Degrees,Longitude coordinate of the city.
3,datetime,Datetime,UTC (ISO 8601),Timestamp of the weather observation in hourly...
4,temperature,Float,°C,Air temperature measured at 2 meters above the...


In [72]:
# Convert list to tabulated format
table = tabulate(data_dict, headers="keys", tablefmt="grid")

# Print the table
print(table)

+----------------+----------+----------------+---------------------------------------------------------------------------+
| Variable       | Type     | Unit           | Description                                                               |
+================+==========+================+===========================================================================+
| city           | String   | N/A            | Name of the city where the weather data was collected.                    |
+----------------+----------+----------------+---------------------------------------------------------------------------+
| latitude       | Float    | Degrees        | Latitude coordinate of the city.                                          |
+----------------+----------+----------------+---------------------------------------------------------------------------+
| longitude      | Float    | Degrees        | Longitude coordinate of the city.                                         |
+---------------

## Source Reference:

Open-Meteo. (n.d.). Open-Meteo Weather API. Retrieved from https://open-meteo.com/